<a href="https://colab.research.google.com/github/marriamaslova/compling_nlp_hse_course/blob/master/notebooks/instruct_fine_tuning/homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнее задание № 10. Генерация текста

### Задание 1 (10 баллов).

В семинаре мы работали с датасетом инструкций alpaca. Он англоязычный. В домашке вам нужно придумать способ создать аналогичный датасет на русском языке и обучить аналогичную модель на этом датасете. 
В качестве итогового результата у вас должна получится модель, которая может связано отвечать на русскоязычные инструкции на русском языке. Приведите как минимум три разных примера. Правильность ответов не так важно, так как вы скорее всего будете использовать небольшие модели, но текст должен быть не рандомным.

Русскоязычный датасет инструкций должен быть больше 5 тысяч примеров. Он может быть основнован на alpaca (например, вы можете просто прогнать все через какую-нибудь переводную модель на huggingface или даже google translate). Или вы можете придумать способ создать аналогичный датасет каким-то другим способом (переделать открытые датасеты с помощью правил). Датасет может быть не уникальным, можно скооперироваться с одногруппниками и сделать один датасет на всех.

Вы можете попробовать дообучать любую небольшую decoder-only модель. Скорее всего лучше всего будут работать модели, изначально обученные на русском языке (rugpt например). Но возможно даже модели вроде opt можно будет дообучить на русскоязычных инструкциях.

Это задание гораздно менее определенное, по сравнению с предыдущими. Поэтому не стесняйтесь задавать дополнительные вопросы в чате или лично, если у вас возникнут трудности.


In [ ]:
%pip install pandas transformers tokenizers datasets xformers

In [ ]:
!pip install sentencepiece
!pip install sacremoses

In [ ]:
import pandas as pd
import numpy as np
import torch
import json

In [ ]:
import copy
import logging
from dataclasses import dataclass, field
from typing import Optional, Dict, Sequence
import json
import torch
import transformers
from torch.utils.data import Dataset
from transformers import Trainer
import sentencepiece
import sacremoses
from transformers import pipeline

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModel

In [ ]:
from transformers import FSMTForConditionalGeneration, FSMTTokenizer

Перевод инструкций на русский язык (по факту, для лучшего контроля за процессом я разделила большой файл с инструкциями и переводила по частям, а в конце соединила результаты)

In [ ]:
tr_model_name = "Helsinki-NLP/opus-mt-en-ru" 

In [ ]:
tr_tokenizer = AutoTokenizer.from_pretrained(tr_model_name)
tr_model = AutoModelForSeq2SeqLM.from_pretrained(tr_model_name) 

In [ ]:
with open('/content/alpaca_cut_data.json', 'r') as alpaca_file:
  data = json.load(alpaca_file)

In [ ]:
def translation(inp):
  input_ids = tr_tokenizer(inp, return_tensors="pt").input_ids
  outputs = tr_model.generate(input_ids=input_ids, num_beams=5, num_return_sequences=1)
  value_new = ''.join(tr_tokenizer.batch_decode(outputs, skip_special_tokens=True))
  return value_new

In [ ]:
instuctions = []
for elem in data:
  for key, value in elem.items():
    if key == 'instruction':
      if value != '':
        inp = value
        value_inst = translation(inp)
      else:
        value_inst = ''
    if key == 'input':
      if value != '':
        inp = value
        value_inp = translation(inp)
      else:
        value_inp = ''
    if key == 'output':
      if value != '':
        inp = value
        value_out = translation(inp)
      else:
        value_out = ''
  elem['instruction'] = value_inst
  elem['input'] = value_inp
  elem['output'] = value_out
  instuctions.append(elem)

In [ ]:
with open('alpaca_cut_rus.json', 'w') as alpaca_file_rus:
  json.dump(instuctions, alpaca_file_rus)

Обучение на инструкциях

In [ ]:
data_alpaca = json.load(open('alpaca_cut_rus.json'))

In [ ]:
data_alpaca[:30]

[{'instruction': 'Дай три чаевых за то, чтобы оставаться здоровой.',
  'input': '',
  'output': '2. Регулярное упражнение, с тем чтобы ваше тело было активным и сильным. 3. Успокойтесь и сохраняйте постоянный график сна.'},
 {'instruction': 'Какие три основных цвета?',
  'input': '',
  'output': 'Три основных цвета красные, синие и желтые.'},
 {'instruction': 'Опишите структуру атома.',
  'input': '',
  'output': 'Атом состоит из ядра, содержащего протоны и нейтроны, окруженного электронами, перемещающимися по орбитам вокруг ядра. Протоны и нейтроны имеют положительный заряд, в то время как электроны имеют отрицательный заряд, что приводит к общему нейтральному атому. Количество частиц определяет атомное число и тип атома.'},
 {'instruction': 'Как мы можем уменьшить загрязнение воздуха?',
  'input': '',
  'output': 'Кроме того, отдельные лица могут внести свой вклад в сокращение загрязнения воздуха путем сокращения использования легковых автомобилей, предотвращения сжигания таких матер

In [ ]:
IGNORE_INDEX = -100
DEFAULT_PAD_TOKEN = "[PAD]"
DEFAULT_EOS_TOKEN = "</s>"
DEFAULT_BOS_TOKEN = "</s>"
DEFAULT_UNK_TOKEN = "</s>"
PROMPT_DICT = {
    "prompt_input": (
        "Below is an instruction that describes a task, paired with an input that provides further context. "
        "Write a response that appropriately completes the request.\n\n"
        "### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:"
    ),
    "prompt_no_input": (
        "Below is an instruction that describes a task. "
        "Write a response that appropriately completes the request.\n\n"
        "### Instruction:\n{instruction}\n\n### Response:"
    ),
}

In [ ]:
def _tokenize_fn(strings: Sequence[str], tokenizer: transformers.PreTrainedTokenizer) -> Dict:
    """Tokenize a list of strings."""
    tokenized_list = [
        tokenizer(
            text,
            return_tensors="pt",
            # padding="max_length",
            max_length=tokenizer.model_max_length,
            truncation=True,
          
        )
        for text in strings
    ]
    input_ids = labels = [tokenized.input_ids[0] for tokenized in tokenized_list]
    input_ids_lens = labels_lens = [
        tokenized.input_ids.ne(tokenizer.pad_token_id).sum().item() for tokenized in tokenized_list
    ]
    return dict(
        input_ids=input_ids,
        labels=labels,
        input_ids_lens=input_ids_lens,
        labels_lens=labels_lens,
    )

In [ ]:
def preprocess(
    sources: Sequence[str],
    targets: Sequence[str],
    tokenizer: transformers.PreTrainedTokenizer,
) -> Dict:
    """Preprocess the data by tokenizing."""
    examples = [s + t for s, t in zip(sources, targets)]
    examples_tokenized, sources_tokenized = [_tokenize_fn(strings, tokenizer) for strings in (examples, sources)]
    input_ids = examples_tokenized["input_ids"]
    labels = copy.deepcopy(input_ids)
    for label, source_len in zip(labels, sources_tokenized["input_ids_lens"]):
        label[:source_len] = IGNORE_INDEX
    return dict(input_ids=input_ids, labels=labels)

In [ ]:
class SupervisedDataset(Dataset):
    """Dataset for supervised fine-tuning."""

    def __init__(self, data_path: str, tokenizer: transformers.PreTrainedTokenizer):
        super(SupervisedDataset, self).__init__()
        logging.warning("Loading data...")
        list_data_dict = json.load(open(data_path))

        logging.warning("Formatting inputs...")
        prompt_input, prompt_no_input = PROMPT_DICT["prompt_input"], PROMPT_DICT["prompt_no_input"]
        sources = [
            prompt_input.format_map(example) if example.get("input", "") != "" else prompt_no_input.format_map(example)
            for example in list_data_dict
        ]
        targets = [f"{example['output']}{tokenizer.eos_token}" for example in list_data_dict]

        logging.warning("Tokenizing inputs... This may take some time...")
        data_dict = preprocess(sources, targets, tokenizer)

        self.input_ids = data_dict["input_ids"]
        self.labels = data_dict["labels"]

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, i) -> Dict[str, torch.Tensor]:
        return dict(input_ids=self.input_ids[i], labels=self.labels[i])


@dataclass
class DataCollatorForSupervisedDataset(object):
    """Collate examples for supervised fine-tuning."""

    tokenizer: transformers.PreTrainedTokenizer

    def __call__(self, instances: Sequence[Dict]) -> Dict[str, torch.Tensor]:
        input_ids, labels = tuple([instance[key] for instance in instances] for key in ("input_ids", "labels"))
        input_ids = torch.nn.utils.rnn.pad_sequence(
            input_ids, batch_first=True, padding_value=self.tokenizer.pad_token_id
        )
        labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=IGNORE_INDEX)
        return dict(
            input_ids=input_ids,
            labels=labels,
            attention_mask=input_ids.ne(self.tokenizer.pad_token_id),
        )

In [ ]:
model_name = "facebook/xglm-564M"
model = transformers.AutoModelForCausalLM.from_pretrained(
        model_name,
        max_length=100,
        cache_dir="huggingface_cache",
    )
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_name,
    cache_dir="huggingface_cache",
    model_max_length=100,
    padding_side="right",
    use_fast=False,
    truncation=True
    )

In [ ]:
d = json.load(open('alpaca_cut_rus.json'))

In [ ]:
json.dump(d, open('alpaca_c.json', 'w'),  indent=4, ensure_ascii=False)

In [ ]:
train_dataset = SupervisedDataset(tokenizer=tokenizer, data_path="alpaca_cut_rus.json", )
data_collator = DataCollatorForSupervisedDataset(tokenizer=tokenizer)

In [ ]:
import torch
dataloader = torch.utils.data.DataLoader(
            train_dataset,
            batch_size=1,
            collate_fn=data_collator,
            shuffle=True,
            drop_last=True,
        )

In [ ]:
for b in dataloader:
    break

In [ ]:
len(b['input_ids'][0])

84

In [ ]:
train_args = transformers.TrainingArguments(learning_rate=1e-5, 
                 num_train_epochs=1,
                 per_device_train_batch_size=5,
                 gradient_accumulation_steps=1,
                 evaluation_strategy='no',
                 weight_decay=0.,
                 warmup_ratio=0.03,
                 lr_scheduler_type="cosine",
                 save_strategy='no',
                 logging_steps=1000,
                 output_dir="opt125_instruct_ft",
                 fp16=True)

In [ ]:
trainer = Trainer(model=model, 
                 tokenizer=tokenizer, 
                 args=train_args,
                 train_dataset=train_dataset, 
                 eval_dataset=None, 
                 data_collator=data_collator)

In [ ]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
1000,3.317300


TrainOutput(global_step=1001, training_loss=3.316117494613617, metrics={'train_runtime': 405.932, 'train_samples_per_second': 12.322, 'train_steps_per_second': 2.466, 'total_flos': 902952830545920.0, 'train_loss': 3.316117494613617, 'epoch': 1.0})

In [ ]:
trainer.save_model('xglm_ft_02')

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
MODEL_NAME = 'xglm_ft_02'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, model_max_length=512, max_length=512)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, max_length=512)

In [ ]:
def predict_for_instruction(instruction, text, model):
    text = text.replace('\n', ' ')
    prompt = ("Below is an instruction that describes a task, paired with an input that provides further context. "
              "Write a response that appropriately completes the request.\n\n"
              f"### Instruction:\n{instruction}\n\n### Input:\n{text}\n\n### Response:")

    inputs = tokenizer([prompt], 
                        return_tensors="pt", padding=True)

    output_sequences = model.generate(
        # this parameters are also important but you can read about them in the docs and just try changing them
        num_beams=1,
#         temperature=0.4,
    #     max_length=100,
        max_new_tokens=20,
    #     no_repeat_ngram_size=4,
    #     repetition_penalty= 5.0,
    #     length_penalty=0.01,
    #     early_stopping=True,
    #     do_sample=True, 
    #     top_k=30, 
    #     top_p=0.8, 
        early_stopping=True,
    #     num_return_sequences=3,
        num_return_sequences= 1,
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        do_sample=False,  # disable sampling to test if batching affects output
    )
    summaries = tokenizer.batch_decode(output_sequences[:,len(inputs[0]):], skip_special_tokens=True)
    return summaries[0]

In [ ]:
text = """
«Ливерпуль» выбыл из борьбы за Джуда Беллингема. 
По данным The Telegraph, мерсисайдцы больше не планируют приобретать хавбека «Боруссии» Дортмунд из-за его цены. На трансфер ушло бы свыше 115 млн фунтов (без учета высокой зарплаты) – большая часть летнего бюджета. 
Вместо этого «Ливерпуль» сосредоточится на том, чтобы усилить каждую линию, требующую этого. Главный тренер Юрген Клопп поддерживает решение руководства. Немецкий специалист хочет, чтобы результаты работы по укреплению состава были видны как можно раньше, и потому менеджеры не хотят участвовать в длительной борьбе за игрока, который в итоге вообще может остаться в Бундеслиге еще на год. 
В числе претендентов на Беллингема называются «Манчестер Юнайтед», «Манчестер Сити», «ПСЖ», «Реал» и «Челси».
"""

In [ ]:
instruction = "Перескажите содержание этого текста."
predict_for_instruction(instruction, text, model)

'Ливерпуль выбыл из борьбы за Джуда Беллингема.'

In [ ]:
instruction = "Напиши заголовок для текста."
predict_for_instruction(instruction, text, model)

'Ливерпуль выбыл из борьбы за Джуда Беллингема.'

In [ ]:
text_1 = """
Владельцев «Манчестер Юнайтед» не устроили заявки на приобретение клуба.
Семья Глейзеров считает, что суммы полученных ими предложений недостаточно велики, сообщает журналист The Independent Мигель Дилэйни.
Претендентам на покупку «Юнайтед» предложат улучшить заявки в третьем раунде переговоров, который пройдет в конце апреля.
Ранее сообщалось, что Глейзеры хотят получить 6 миллиардов фунтов за продажу «Манчестер Юнайтед».
Катарский шейх Джасим бин Хамад Аль Тани и владелец нефтяной компании INEOS англичанин Джим Рэтклифф предложили за клуб АПЛ около 5 миллиардов фунтов, финский бизнесмен и бывший топ-менеджер Nokia Томас Зиллиакус – меньше 4 млрд.
"""

In [ ]:
instruction = "Напиши краткое содержание текста."
predict_for_instruction(instruction, text_1, model)

'Семья Глейзеров считает, что суммы полученных ими предложений недостаточно велики.'

In [ ]:
instruction = "Напиши заголовок для текста."
predict_for_instruction(instruction, text_1, model)

'Семья Глейзеров считает, что суммы полученных ими предложений недостаточно велики.'

Если как-то видоизменять запросы, оставаясь в контексте суммаризации и "осмысления" материала, то модель даст такие же ответы, как и выше. Я пробовала другие тексты и видела варианты ответов, больше похожие именно на суммаризацию и с бОльшим "творчеством", но то, что есть выше, как-то покрасивее.

Мне не очень нравится такое однообразие ответов. Но, во-первых, с ними не поспоришь, они более чем адекватные. Во-вторых, есть подозрение, что 5000 инструкций - маловато для более качественной работы с текстом. Все же для оригинальной alpaca инструкций было в разы больше, и как знать, может быть многое, подходящее для работы с текстом, не попало в мою выборку. 